In [4]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm 
import datetime, time, gc 
# from utils import distance, haversine, standard, pad_seq 
from scipy.stats import skew, kurtosis
from zipfile import ZipFile
from collections import Counter 
from sklearn.metrics import roc_auc_score as auc
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score as auc
# import tables

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
!pip install tables

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 4.3 MB 8.2 MB/s eta 0:00:01
     |████████████████████████████████| 162 kB 14.9 MB/s eta 0:00:01


In [3]:
# !pip install pandas==0.23.0
!pip install lightgbm

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 1.2 MB 7.6 MB/s eta 0:00:01


In [5]:
# train = pd.read_pickle("data/train_fea.pkl")
# test = pd.read_pickle("data/test_fea.pkl")

train = pd.read_hdf("data/train_fea.hdf", key='df')
test = pd.read_hdf("data/test_fea.hdf", key='df')

print(train.shape, test.shape)

train = train.sample(35000000)
train = train.reset_index(drop=True)
train.head()

(41907133, 106) (1000000, 105)


,adv_id,adv_prim_id,age,app_first_class,app_score,app_second_class,career,city,city_rank,communication_avgonline_30d,communication_onlinerate,consume_purchase,creat_type_cd,dev_id,device_name,device_price,device_size,emui_dev,gender,his_app_size,his_on_shelf_time,indu_name,inter_type_cd,label,list_time,membership_life_duration,net_type,residence,slot_id,spread_app_id,tags,task_id,uid,up_life_duration,up_membership_grade,uid_task_id_nunique,uid_task_id_count,uid_task_id_rate,task_id_uid_nunique,task_id_uid_count,uid_adv_id_nunique,uid_adv_id_count,uid_adv_id_rate,adv_id_uid_nunique,adv_id_uid_count,uid_spread_app_id_nunique,uid_spread_app_id_count,uid_spread_app_id_rate,adv_id_count_fea,adv_prim_id_count_fea,task_id_count_fea,uid_count_fea,city_count_fea,uid_ctr,task_id_ctr,adv_id_ctr,uid_task_id_ctr,uid_adv_id_ctr,uid_spread_app_id_ctr,uid_adv_prim_id_ctr,uid_dev_id_ctr,uid_app_second_class_ctr,age_task_id_ctr,age_adv_id_ctr,age_spread_app_id_ctr,age_adv_prim_id_ctr,age_dev_id_ctr,age_app_second_class_ctr,gender_task_id_ctr,gender_adv_id_ctr,gender_spread_app_id_ctr,gender_adv_prim_id_ctr,gender_dev_id_ctr,gender_app_second_class_ctr,city_task_id_ctr,city_adv_id_ctr,city_spread_app_id_ctr,city_adv_prim_id_ctr,city_dev_id_ctr,city_app_second_class_ctr,career_task_id_ctr,career_adv_id_ctr,career_spread_app_id_ctr,career_adv_prim_id_ctr,career_dev_id_ctr,career_app_second_class_ctr,cnt_click_of_uid_and_task_id,cnt_click_of_uid_and_adv_id,cnt_click_of_uid_and_spread_app_id,cnt_click_of_uid_and_adv_prim_id,cnt_click_of_age_and_task_id,cnt_click_of_age_and_adv_id,cnt_click_of_age_and_spread_app_id,cnt_click_of_age_and_adv_prim_id,cnt_click_of_gender_and_task_id,cnt_click_of_gender_and_adv_id,cnt_click_of_gender_and_spread_app_id,cnt_click_of_gender_and_adv_prim_id,cnt_click_of_city_and_task_id,cnt_click_of_city_and_adv_id,cnt_click_of_city_and_spread_app_id,cnt_click_of_city_and_adv_prim_id,cnt_click_of_career_and_task_id,cnt_click_of_career_and_adv_id,cnt_click_of_career_and_spread_app_id,cnt_click_of_career_and_adv_prim_id
0,295,64,7,2,2,14,7,4,4,10,18,0,5,22,53,2,141,14,0,9,3,6,3,0,10,-1,0,35,8,31,27,945,298144,-1,0,17,25,0.680176,136858,206905,17,25,0.680176,136858,206905,11,25,0.439941,0.368408,0.502441,0.368408,0.015121,0.129639,0.062317,0.045319,0.045319,NaN,NaN,NaN,NaN,0.019547,0.022064,0.057465,0.057465,0.049438,0.044434,0.054626,0.055939,0.045837,0.045837,0.029770,0.027161,0.033325,0.038208,0.042236,0.042236,0.030823,0.028748,0.033142,0.038513,0.049469,0.049469,0.036438,0.033295,0.040924,0.043427,0,0,0,0,12,12,33,30,36,36,66,50,3,3,4,7,32,32,56,51
1,4432,73,4,2,2,14,2,52,3,12,21,0,5,40,44,4,141,20,2,13,3,6,3,0,8,-1,0,3,6,19,27,1269,643652,-1,0,180,314,0.573242,90739,155926,182,314,0.579590,90739,155926,20,314,0.063721,0.277588,0.317139,0.277588,0.197266,0.055115,0.005829,0.081848,0.081848,0.002039,0.002008,0.001637,0.001706,0.001496,0.001151,0.081848,0.081848,0.028870,0.030136,0.026489,0.032898,0.093079,0.093079,0.029449,0.029709,0.030823,0.036652,0.066040,0.066162,0.023666,0.022736,0.030670,0.040894,0.083679,0.083679,0.028198,0.028549,0.027924,0.033875,1,1,7,6,22,22,23,25,9,9,12,10,0,0,1,2,26,26,32,32
2,2472,53,5,2,2,10,7,130,5,11,21,0,5,40,21,5,141,20,0,14,3,6,3,0,4,-1,0,9,6,44,24,1827,851184,-1,0,13,14,0.928711,116790,169950,13,14,0.928711,116790,169950,4,14,0.285645,0.302490,0.687988,0.302490,0.008194,0.416016,0.012047,0.062988,0.062988,NaN,NaN,0.007988,0.007168,0.009232,0.011276,0.067139,0.067139,0.024994,0.024994,0.026947,0.031799,0.066895,0.066895,0.035187,0.035187,0.034149,0.042267,0.059204,0.059204,0.032318,0.032318,0.032196,0.037598,0.062469,0.062469,0.044830,0.044830,0.041901,0.052917,0,0,1,1,27,27,52,70,33,33,69,69,6,6,16,33,21,21,57,70
3,4997,3,7,2,2,10,7,78,3,13,24,0,5,22,27,4,162,20,0,6,3,6,3,0,4,-1,0,28,5,37,24,4472,610562,-1,0,131,555,0.236084,90436,191561,131,555,0.236084,90436,191561,12,555,0.021622,0.341064,0.170532,0.341064,0.349121,0.038422,0.009705,0.029404,0.029404,0.001586,0.001565,0.047455,0.04818

In [10]:
ctr_col = ['uid_ctr', 'task_id_ctr', 'adv_id_ctr', 'uid_task_id_ctr', 'uid_adv_id_ctr', 'uid_spread_app_id_ctr', 
           'uid_adv_prim_id_ctr', 'uid_dev_id_ctr', 'uid_app_second_class_ctr', 'age_task_id_ctr', 'age_adv_id_ctr', 
           'age_spread_app_id_ctr', 'age_adv_prim_id_ctr', 'age_dev_id_ctr', 'age_app_second_class_ctr',
           'gender_task_id_ctr',
           'gender_adv_id_ctr', 'gender_spread_app_id_ctr', 'gender_adv_prim_id_ctr', 'gender_dev_id_ctr', 
           'gender_app_second_class_ctr', 'city_task_id_ctr', 'city_adv_id_ctr', 'city_spread_app_id_ctr',
           'city_adv_prim_id_ctr', 'city_dev_id_ctr', 'city_app_second_class_ctr', 'career_task_id_ctr', 
           'career_adv_id_ctr', 
           'career_spread_app_id_ctr', 'career_adv_prim_id_ctr', 'career_dev_id_ctr', 'career_app_second_class_ctr']
print(train[ctr_col].count())
print()
print(test[ctr_col].count())

uid_ctr                        29899442
task_id_ctr                    29999869
adv_id_ctr                     29999850
uid_task_id_ctr                13028711
uid_adv_id_ctr                 12870845
uid_spread_app_id_ctr          25389264
uid_adv_prim_id_ctr            24442364
uid_dev_id_ctr                 26469931
uid_app_second_class_ctr       27560514
age_task_id_ctr                29995532
age_adv_id_ctr                 29993344
age_spread_app_id_ctr          29999984
age_adv_prim_id_ctr            29999980
age_dev_id_ctr                 29999980
age_app_second_class_ctr       29999997
gender_task_id_ctr             29998538
gender_adv_id_ctr              29998339
gender_spread_app_id_ctr       29999983
gender_adv_prim_id_ctr         29999972
gender_dev_id_ctr              29999980
gender_app_second_class_ctr    29999997
city_task_id_ctr               29677588
city_adv_id_ctr                29647735
city_spread_app_id_ctr         29997711
city_adv_prim_id_ctr           29997107


In [6]:
cate_cols = ['uid',
       'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
       'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
       'app_first_class', 'app_second_class', 'city',
       'device_name', 'career', 'gender', 'net_type',
       'residence', 'consume_purchase', 
       'up_membership_grade', 'indu_name',
]

use_cols = list( set(train.columns) - set(['label', 'cnt', 'random_sector',
        'uid_task_id_ctr', 'uid_adv_id_ctr', 
#         'uid_spread_app_id_ctr', 'uid_adv_prim_id_ctr', 'uid_dev_id_ctr',
#         'uid_app_second_class_ctr', 
        'his_on_shelf_time', 
#         'gender_adv_id_ctr', 'gender_adv_id_ctr', 'age_task_id_ctr', 'age_adv_id_ctr',
#         'career_adv_id_ctr', 'career_task_id_ctr', 'city_adv_id_ctr', 'city_task_id_ctr', 
]))

In [7]:
# print(train[cate_cols].nunique())

In [8]:
print(len(use_cols), len(cate_cols))

102 21


In [9]:
# Log
file_name = datetime.date.today().strftime('%m%d')+"_{}.log".format("lgb_base")
def write_log(w):
    t0 = datetime.datetime.now().strftime('%H:%M:%S')
    info = "{} : {}\n".format(t0, w)
    print(info)
    with open(file_name, 'a') as f:
        f.write(info)
        f.write("-"*80+"\n")

In [10]:
# 采用xdeepfm模型
!free -m

              total        used        free      shared  buff/cache   available
Mem:          96166       16447       56632          18       23086       79554
Swap:             0           0           0


In [11]:
%%time


def build_model(train, test, label, seed=2020, is_shuffle=True):
    imp = pd.DataFrame()  # 特征重要性
    imp['feat'] = use_cols
    
    train_pred = np.zeros((train.shape[0], ))
    test_pred = np.zeros((test.shape[0], ))
    n_splits = 5
    # Kfold
    # fold = GroupKFold(n_splits=n_splits)
    fold = KFold(n_splits=n_splits, shuffle=is_shuffle, random_state=seed)
    # groups = train_df['loadingOrder'].values
    kf_way = fold.split(train)
    # params
    params = {
        'learning_rate': 0.05,
        'objective': 'binary',
        'metric': ['binary_logloss', 'auc'], 
        'boosting_type': 'gbdt',
        'num_leaves': 256,
        'feature_fraction': 0.90,
        'bagging_fraction': 0.85,
        'bagging_freq': 3,
        'seed': 8,
        'bagging_seed': 1,
        'feature_fraction_seed': 7,
        'min_data_in_leaf': 20,
        'nthread': 45,
        'verbose': 1,
    }
    # train
    for n_fold, (train_idx, valid_idx) in enumerate(kf_way, start=1):
        write_log("fold {}".format(n_fold))
  
        # 数据加载
        n_train = lgb.Dataset(train.iloc[train_idx], label=label[train_idx], 
                             # free_raw_data=False
                             )
        n_valid = lgb.Dataset(train.iloc[valid_idx], label=label[valid_idx], 
                              #free_raw_data=False
                             )

        clf = lgb.train(
            params=params,
            train_set=n_train,
            categorical_feature=cate_cols,
            num_boost_round=10000,
            valid_sets=[n_train, n_valid],
            early_stopping_rounds=30,
            verbose_eval=20,
#             feval=mse_score_eval
        )
        
        clf.save_model("data/save_model/lgb_model_fold_{}.txt".format(n_fold))
        train_pred[valid_idx] = clf.predict(train.iloc[valid_idx], num_iteration=clf.best_iteration)
        
        write_log("val_auc = {}".format(auc(label[valid_idx], train_pred[valid_idx])))
        
        test_pred += clf.predict(test, num_iteration=clf.best_iteration) / fold.n_splits

        imp['gain' + str(n_fold + 1)] = clf.feature_importance(importance_type='gain')
        imp['split' + str(n_fold + 1)] = clf.feature_importance(importance_type='split')
        break
    
    write_log("train mse: {}".format(auc(label, train_pred)))
    result = pd.DataFrame({
        'id': np.load("data/test_id.npy"), 
        'probability': test_pred * 5, 
    })
    return result, imp

result, imp = build_model(train[use_cols], test[use_cols], train['label'].values, is_shuffle=True)

result.to_pickle("data/result.pkl")
imp.to_pickle("data/fea.pkl")

12:38:08 : fold 1



/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['adv_id', 'adv_prim_id', 'app_first_class', 'app_second_class', 'career', 'city', 'consume_purchase', 'creat_type_cd', 'dev_id', 'device_name', 'gender', 'indu_name', 'inter_type_cd', 'net_type', 'residence', 'slot_id', 'spread_app_id', 'tags', 'task_id', 'uid', 'up_membership_grade']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 30 rounds
[20]	training's binary_logloss: 0.116094	training's auc: 0.842256	valid_1's binary_logloss: 0.116696	valid_1's auc: 0.839383
[40]	training's binary_logloss: 0.111689	training's auc: 0.848391	valid_1's binary_logloss: 0.112748	valid_1's auc: 0.842707
[60]	training's binary_logloss: 0.109886	training's auc: 0.853903	valid_1's binary_logloss: 0.1115	valid_1's auc: 0.845391
[80]	training's binary_logloss: 0.108775	training's auc: 0.858802	valid_1's binary_logloss: 0.110996	valid_1's auc: 0.846963
[100]	training's binary_logloss: 0.107929	training's auc: 0.862854	valid_1's binary_logloss: 0.110764	valid_1's auc: 0.847725
[120]	training's binary_logloss: 0.107196	training's auc: 0.866383	valid_1's binary_logloss: 0.11061	valid_1's auc: 0.84824
[140]	training's binary_logloss: 0.106525	training's auc: 0.869554	valid_1's binary_logloss: 0.110504	valid_1's auc: 0.848558
[160]	training's binary_logloss: 0.10594	training's auc: 0.87247	

In [11]:
result['probability'] = result['probability'] * 0.2

In [12]:
result

,id,probability
0,1,0.003594
1,2,0.249894
2,3,0.414042
3,4,0.034582
4,5,0.082098
...,...,...
999995,999996,0.013060
999996,999997,0.012009
999997,999998,0.138414
999998,999999,0.020005


In [13]:
result.to_csv("submission.csv", index=False)

In [6]:
imp = pd.read_pickle("data/fea.pkl")

imp['gain'] = imp[[f for f in imp.columns if 'gain' in f]].sum(axis=1)/10
imp['split'] = imp[[f for f in imp.columns if 'split' in f]].sum(axis=1)/10

imp = imp[['feat', 'gain', 'split']]
imp = imp.sort_values(by=['gain'], ascending=False)
imp = imp.reset_index(drop=True)
# imp.to_pickle("imp_2.pkl")
imp

,feat,gain,split
0,uid_adv_id_count,2.391529e+06,272.1
1,uid_adv_id_nunique,1.666425e+06,108.3
2,uid_ctr,8.112983e+05,911.1
3,task_id,5.197752e+05,4007.1
4,adv_id,3.730890e+05,3727.0
5,slot_id,3.507676e+05,303.2
6,uid_task_id_nunique,2.634810e+05,116.3
7,city,2.377588e+05,3435.2
8,adv_prim_id,1.704940e+05,294.8
9,age_adv_id_ctr,1.505979e+05,50.3
